In [70]:
import numpy as np
from sklearn.metrics import roc_curve

from collections import defaultdict

#"asvspoof_2021_DF", "asvspoof2019_la", "asvspoof2021_la", "audiodec_24k_320d", 
#                 "av_deepfake_1m", "descript-audio-codec-16khz", "emofake", "encodec_24khz", "fakeavceleb_all",
#                 "funcodec-funcodec_en_libritts-16k-nq32ds320", "speech_tokenizer", "mlaad","scenefake","release_in_the_wild", 
#                 "vctk", "ami_ihm", "ami_sdm", "librispeech_test_clean", "librispeech_test_other",
#                 "llamapartialspoof_r01tts0a"
dataset_configs = {
    #"academicodec_hifi_16k_320d": {"only_include_id_patterns":["p227","p228"]},
    "asvspoof_2021_DF": {}
}
model_name = "conformer-based-classifier-for-anti-spoofing"

id2meta = defaultdict(lambda: defaultdict( lambda: defaultdict(dict) )) # 3-levels nested default dict
id2count = defaultdict(lambda: defaultdict( lambda: defaultdict(dict) )) # 3-levels nested default dict
for dataset_name, config in dataset_configs.items():
    
    with open(f"../eer_files/{model_name}/scores/{dataset_name}.txt", 'r') as rf:
        lines = rf.readlines()
        for i, line in enumerate(lines):
            if line == "\n": continue
            split = line.strip().split(' ')
            id_ = split[0]
            score = float(split[1])
            
            #if "DF_E_2101080" in line: continue
            if config.get("only_include_id_patterns", None) is not None:
                if all([_ not in id_ for _ in config["only_include_id_patterns"]]):
                    continue

            id2meta[model_name][dataset_name][id_]["score"] = score

            if " eval " in line or " eval\n" in line: assert split != "eval"
    
    with open(f"../eer_files/{model_name}/keys/{dataset_name}.txt", 'r') as rf:
        lines = rf.readlines()
        for i, line in enumerate(lines):
            if line == "\n": continue
            split = line.strip().split(' ')
            id_ = split[1]
            data_type = split[7]
            label = split[5]
            codec = split[2]
            synthesizer = split[4]

            if config.get("only_include_id_patterns", None) is not None:
                if all([_ not in id_ for _ in config["only_include_id_patterns"]]):
                    continue
            
            if data_type == "eval":
                id2meta[model_name][dataset_name][id_]["label"] = label
                id2meta[model_name][dataset_name][id_]["codec"] = codec
                if synthesizer != "-":
                    id2meta[model_name][dataset_name][id_]["synthesizer"] = synthesizer
                    id2count[model_name][dataset_name]["synthesizer"][synthesizer] = \
                        id2count[model_name][dataset_name]["synthesizer"].get(synthesizer, 0) + 1
                if codec != "-" and codec != "nocodec":
                    id2count[model_name][dataset_name]["codec"][codec] = \
                        id2count[model_name][dataset_name]["codec"].get(codec, 0) + 1

            if " eval " in line or " eval\n" in line: assert split != "eval"
            
    print(f"Finish reading meta info for {dataset_name}")

Finish reading meta info for asvspoof_2021_DF


In [71]:
id2meta["conformer-based-classifier-for-anti-spoofing"]["asvspoof_2021_DF"]

defaultdict(dict,
            {'DF_E_2000011': {'score': -4.120094299316406},
             'DF_E_2000013': {'score': -3.6416542530059814,
              'label': 'spoof',
              'codec': 'low_m4a',
              'synthesizer': 'Task1-team20'},
             'DF_E_2000024': {'score': -4.402594089508057,
              'label': 'spoof',
              'codec': 'mp3m4a',
              'synthesizer': 'Task2-team12'},
             'DF_E_2000026': {'score': -4.237467288970947,
              'label': 'spoof',
              'codec': 'mp3m4a',
              'synthesizer': 'A09'},
             'DF_E_2000027': {'score': -4.120230674743652,
              'label': 'spoof',
              'codec': 'mp3m4a',
              'synthesizer': 'A12'},
             'DF_E_2000028': {'score': -4.5333685874938965,
              'label': 'spoof',
              'codec': 'low_mp3',
              'synthesizer': 'SPO-N12'},
             'DF_E_2000031': {'score': -4.571272373199463,
              'label': 'spoof',


In [72]:
id2count["conformer-based-classifier-for-anti-spoofing"]["asvspoof_2021_DF"]["synthesizer"]

{'Task1-team20': 3600,
 'Task2-team12': 5400,
 'A09': 5577,
 'A12': 5628,
 'SPO-N12': 5040,
 'A10': 5709,
 'Task2-team29': 5400,
 'SPO-B01': 5040,
 'HUB-N06': 5040,
 'Task1-team09': 3600,
 'SPO-N10': 5040,
 'A17': 5654,
 'Task1-team11': 3600,
 'HUB-N12': 5040,
 'Task2-team16': 5400,
 'SPO-N11': 5040,
 'Task1-team18': 3600,
 'A08': 5740,
 'Task1-team33': 3600,
 'HUB-D04': 5040,
 'Task2-team24': 5400,
 'Task2-team07': 5400,
 'Task2-team02': 5400,
 'HUB-N11': 5040,
 'SPO-N03': 5040,
 'HUB-N10': 5040,
 'Task2-team09': 5400,
 'HUB-N18': 5040,
 'Task1-team19': 3600,
 'A16': 5797,
 'Task1-team23': 3600,
 'SPO-N18': 5040,
 'SPO-N06': 5040,
 'A11': 5788,
 'Task1-team27': 3600,
 'Task2-team11': 5400,
 'SPO-N13': 5040,
 'Task1-team12': 3600,
 'HUB-D02': 5040,
 'A13': 5756,
 'HUB-D03': 5040,
 'Task2-team13': 5400,
 'HUB-N20': 5040,
 'A18': 5578,
 'Task1-team07': 3600,
 'SPO-N05': 5040,
 'HUB-N16': 5040,
 'HUB-N08': 5040,
 'Task1-team01': 3600,
 'A07': 5723,
 'HUB-N07': 5040,
 'Task2-team22': 5400,

In [73]:
def get_subset(bonafide_patterns=[None,None,None], spoof_patterns=[None,None,None]):
    y, score = [], []
    counts = [0,0] # spoof count, bona fide count
    for model in id2meta.keys():
        if spoof_patterns[0] is None or spoof_patterns[0] in model:
            for dataset in id2meta[model].keys():
                if spoof_patterns[1] is None or spoof_patterns[1] in dataset:
                    for id_ in id2meta[model][dataset].keys():
                        if spoof_patterns[2] is None or spoof_patterns[2] in id_:
                            if id2meta[model][dataset][id_].get("label", None) == "spoof":
                                try:
                                    score.append(id2meta[model][dataset][id_]["score"])
                                    y.append(0)
                                    counts[0] += 1
                                except:
                                    print(id_)
        if bonafide_patterns[0] is None or bonafide_patterns[0] in model:
            for dataset in id2meta[model].keys():
                if bonafide_patterns[1] is None or bonafide_patterns[1] in dataset:
                    for id_ in id2meta[model][dataset].keys():
                        if bonafide_patterns[2] is None or bonafide_patterns[2] in dataset:
                            if id2meta[model][dataset][id_].get("label", None) == "bonafide":
                                try:
                                    score.append(id2meta[model][dataset][id_]["score"])
                                    y.append(1)
                                    counts[1] += 1
                                except:
                                    print(id_)
    print(counts)
    return y, score

def get_eer(y, score):
    fpr, tpr, threshold = roc_curve(y, score, pos_label=1)
    fnr = 1 - tpr
    eer_threshold = threshold[np.nanargmin(np.absolute((fnr - fpr)))]
    EER = fpr[np.nanargmin(np.absolute((fnr - fpr)))]
    eer_precision = fnr[np.nanargmin(np.absolute((fnr - fpr)))] - fpr[np.nanargmin(np.absolute((fnr - fpr)))]
    assert eer_precision < 0.02, eer_precision
    print("eer threshold =", eer_threshold)
    return EER

In [74]:
get_eer(*get_subset())

DF_E_2101080
[519058, 14869]
eer threshold = -3.6213181018829346


0.03469361805424442